In [ ]:
import streamlit as st
import zipfile
import os
import pandas as pd
import shutil
import base64
from docaiprocess import process_document  # Import the function you've developed
from sqlalchemy import create_engine, inspect, text


# Create a SQLAlchemy engine connected to a SQLite database
engine = create_engine('sqlite:///databases/CFE_Recibos_DB.sqlite')

def front_page():

    st.markdown("# Saturnia Receipts App", unsafe_allow_html=True)
    st.markdown(
        """
        <style>
        h1{
        font-size:60px !important;
        color:#DFFF00;
        }
        </style>
        """, 
        unsafe_allow_html=True
    )

    st.markdown("""

    Put here your client's recipts and summarize them into an excel! 

    To introduce a .zip file:

    1. Click on the 'Browse files' button below.
    2. Navigate to the location of your .zip file in the file dialog.
    3. Select the .zip file and click 'Open'.
    """)

uploaded_file = st.file_uploader("Upload file", type=['zip'])

# if os.path.exists('recibos') and os.path.isdir('recibos'):
#     shutil.rmtree('recibos')  # Deletes the directory

if uploaded_file is not None:
    with zipfile.ZipFile(uploaded_file, 'r') as zip_ref:
        for member in zip_ref.namelist():
            # This will exclude any directories
                if not member.endswith('/'):
                    zip_ref.extract(member, 'recibos')

def process_and_send_files():
    fields = {}
    files = os.listdir('recibos')
    with st.spinner('Processing...'):
        for i, filename in enumerate(files):
            if filename.endswith('.pdf'):
                name = filename.split(".")[0]
                file_path = os.path.join('recibos', filename)
                result = process_document(file_path)
                fields[name] = result

    df = pd.DataFrame.from_dict(fields, orient='index')
    csv = df.to_csv(index=True, index_label="recibo").encode()

    st.success('Done!')
    st.download_button(label="Download Results as CSV", data=csv, file_name='recibos_data.csv', mime='text/csv')
    return df

def save_data_to_db(engine, df):
    inspector = inspect(engine)
    with engine.connect() as connection:
        if 'recibos' not in inspector.get_table_names():
            df.to_sql('recibos', connection, index=True, index_label="recibo")
        else:
                existing_ids = pd.read_sql('SELECT recibo FROM recibos', connection)
                df_new = df[~df.index.isin(existing_ids.index)]
                if not df_new.empty:
                    df_new.to_sql('recibos', engine, index=True, index_label='recibo', if_exists='append')

    
        result = connection.execute(text("SELECT DISTINCT * FROM recibos LIMIT 10"))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())

        # Query the database to get the first 10 rows from 'recibos' table
        
        st.markdown('Collect your clients data into your database. Until now, these are the first 10 rows that you have collected so far:')
        st.dataframe(df)

In [8]:
documentai.DocumentProcessorServiceClient()

DefaultCredentialsError: File home/miguel/repos/saturnia-recibos/saturnia-recibos-42d2ad3e0669.json was not found.

In [23]:
import pandas as pd

df = pd.read_csv("recibos_data.csv")
df.head()

,Unnamed: 0,due_date,invoice_date,total_amount,invoice_id,supplier_registration,invoice_type,supplier_address,line_item_1,line_item_2,line_item_3,line_item_4,line_item_5,line_item_6,line_item_7,line_item_8
0,recibo_test copy 2.png,12 ABR 22,28 FEB 22 - 31 MAR 22,"1,821,864",999850102730,GDMTH,NaN,ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELS...,"kW punta 1,361","kW intermedia 1,615","kWh punta 94,590","kWh intermedia 423,442","kW base 1,230","kWh base 163,738",CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW ...,kWMax
1,recibo_test copy 4.png,12 ABR 22,28 FEB 22 - 31 MAR 22,"1,821,864",999850102730,GDMTH,NaN,ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELS...,"kW punta 1,361","kW intermedia 1,615","kWh punta 94,590","kWh intermedia 423,442","kW base 1,230","kWh base 163,738",CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW ...,kWMax
2,recibo_test.png,12 ABR 22,28 FEB 22 - 31 MAR 22,"1,821,864",999850102730,GDMTH,NaN,ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELS...,"kW punta 1,361","kW intermedia 1,615","kWh punta 94,590","kWh intermedia 423,442","kW base 1,230","kWh base 163,738",CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW ...,kWMax
3,recibo_test copy.png,12 ABR 22,28 FEB 22 - 31 MAR 22,"1,821,864",999850102730,GDMTH,NaN,ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELS...,"kW punta 1,361","kW intermedia 1,615","kWh punta 94,590","kWh intermedia 423,442","kW base 1,230","kWh base 163,738",CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW ...,kWMax
4,recibo_test copy 3.png,12 ABR 22,28 FEB 22 - 31 MAR 22,"1,821,864",999850102730,GDMTH,NaN,ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELS...,"kW punta 1,361","kW intermedia 1,615","kWh punta 94,590","kWh intermedia 423,442","kW base 1,230","kWh base 163,738",CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW ...,kWMax


In [13]:
import os
import json 
from google.cloud import documentai_v1beta3 as documentai

def process_document(keys: dict, file_path: str, mime_type: str):

    # Use the built-in Invoice Parser processor
    project_id, location, processor_id = keys['project_id'], keys['location'], keys['processor_id']
    
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "keys/saturnia-recibos-42d2ad3e0669.json"
    client = documentai.DocumentProcessorServiceClient()

    # The full resource name of the processor
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    with open(file_path, "rb") as image:
        image_content = image.read()

    document = {"mime_type": mime_type, "content": image_content}

    request = {"name": name, "document": document}

    result = client.process_document(request=request)

    document = result.document
    print("Document processing complete.")
    
    # Extract predicted entities
    predicted_fields = {}
    i = 1
    for entity in document.entities:
        field_name = entity.type_ 

        if field_name == 'line_item':
            field_name += f"_{i}"
            i += 1

        field_value = entity.mention_text
        predicted_fields[field_name] = field_value


    return predicted_fields

def get_text(anchor, document):
    """
    Function to locate a layout token given its layout anchor.
    """
    response = ''
    for segment in anchor.segments:
        start_index = segment.start_index
        end_index = segment.end_index
        response += document.text[start_index:end_index]
    return response.strip()

def get_keys():
    # Read json file:
    file_path = 'keys/saturnia-recibos-42d2ad3e0669.json'
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


In [14]:
keys = get_keys()

In [15]:
# Call the function
predicted_fields= process_document(keys=keys, file_path="recibos/recibo_test.png", mime_type="image/png")


Document processing complete.


In [16]:
predicted_fields

{'due_date': '12 ABR 22',
 'invoice_date': '28 FEB 22 - 31 MAR 22',
 'total_amount': '1,821,864',
 'invoice_id': '999850102730',
 'supplier_registration': 'GDMTH',
 'invoice_type': '',
 'supplier_address': 'ANASTASIO BUSTAMANTE SN\nNTE Y ID DD10E09V ELSTER Y ID DD10E09V ELSTER\nCENTRO SAN NICOLAS\nSAN NICOLAS DE LOS G, N.L.',
 'line_item_1': 'kW punta 1,361',
 'line_item_2': 'kW intermedia 1,615',
 'line_item_3': 'kWh punta 94,590',
 'line_item_4': 'kWh intermedia 423,442',
 'line_item_5': 'kW base 1,230',
 'line_item_6': 'kWh base 163,738',
 'line_item_7': 'CARGA CONECTADA KW 3100 DEMANDA CONTRATADA KW 2991',
 'line_item_8': 'kWMax'}

In [20]:
import re
from typing import Optional

from google.api_core.client_options import ClientOptions
from google.api_core.exceptions import InternalServerError
from google.api_core.exceptions import RetryError
from google.cloud import documentai  # type: ignore
from google.cloud import storage

# TODO(developer): Uncomment these variables before running the sample.
# project_id = 'YOUR_PROJECT_ID'
# location = 'YOUR_PROCESSOR_LOCATION' # Format is 'us' or 'eu'
# processor_id = 'YOUR_PROCESSOR_ID' # Create processor before running sample
# gcs_input_uri = "YOUR_INPUT_URI" # Format: gs://bucket/directory/file.pdf
# input_mime_type = "application/pdf"
# gcs_output_bucket = "YOUR_OUTPUT_BUCKET_NAME" # Format: gs://bucket
# gcs_output_uri_prefix = "YOUR_OUTPUT_URI_PREFIX" # Format: directory/subdirectory/
# field_mask = "text,entities,pages.pageNumber"  # Optional. The fields to return in the Document object.


def batch_process_documents(
    keys: dict,
    gcs_input_uri: str,
    # input_mime_type: str,
    gcs_output_bucket: str,
    gcs_output_uri_prefix: str,
    # field_mask: Optional[str] = None,
    # timeout: int = 400,
) -> None:
    
    location, project_id, processor_id = keys['location'], keys['project_id'], keys['processor_id'] 
    input_mime_type = 'image/png'
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    gcs_document = documentai.GcsDocument(
        gcs_uri=gcs_input_uri, mime_type=input_mime_type
    )

    # Load GCS Input URI into a List of document files
    gcs_documents = documentai.GcsDocuments(documents=[gcs_document])
    input_config = documentai.BatchDocumentsInputConfig(gcs_documents=gcs_documents)

    # NOTE: Alternatively, specify a GCS URI Prefix to process an entire directory
    #
    # gcs_input_uri = "gs://bucket/directory/"
    # gcs_prefix = documentai.GcsPrefix(gcs_uri_prefix=gcs_input_uri)
    # input_config = documentai.BatchDocumentsInputConfig(gcs_prefix=gcs_prefix)
    #

    # Cloud Storage URI for the Output Directory
    # This must end with a trailing forward slash `/`
    destination_uri = f"{gcs_output_bucket}/{gcs_output_uri_prefix}"

    gcs_output_config = documentai.DocumentOutputConfig.GcsOutputConfig(
        gcs_uri=destination_uri
    )

    # Where to write results
    output_config = documentai.DocumentOutputConfig(gcs_output_config=gcs_output_config)

    # The full resource name of the processor, e.g.:
    # projects/project_id/locations/location/processor/processor_id
    name = client.processor_path(project_id, location, processor_id)

    request = documentai.BatchProcessRequest(
        name=name,
        input_documents=input_config,
        document_output_config=output_config,
    )

    # BatchProcess returns a Long Running Operation (LRO)
    operation = client.batch_process_documents(request)

    # Continually polls the operation until it is complete.
    # This could take some time for larger files
    # Format: projects/PROJECT_NUMBER/locations/LOCATION/operations/OPERATION_ID
    try:
        print(f"Waiting for operation {operation.operation.name} to complete...")
        operation.result()
    # Catch exception when operation doesn't finish before timeout
    except (RetryError, InternalServerError) as e:
        print(e.message)

    # NOTE: Can also use callbacks for asynchronous processing
    #
    # def my_callback(future):
    #   result = future.result()
    #
    # operation.add_done_callback(my_callback)

    # Once the operation is complete,
    # get output document information from operation metadata
    metadata = documentai.BatchProcessMetadata(operation.metadata)

    if metadata.state != documentai.BatchProcessMetadata.State.SUCCEEDED:
        raise ValueError(f"Batch Process Failed: {metadata.state_message}")

    storage_client = storage.Client()

    print("Output files:")
    # One process per Input Document
    for process in metadata.individual_process_statuses:
        # output_gcs_destination format: gs://BUCKET/PREFIX/OPERATION_NUMBER/INPUT_FILE_NUMBER/
        # The Cloud Storage API requires the bucket name and URI prefix separately
        matches = re.match(r"localhost://(.*?)/(.*)", process.output_gcs_destination)
        if not matches:
            print(
                "Could not parse output GCS destination:",
                process.output_gcs_destination,
            )
            continue

        output_bucket, output_prefix = matches.groups()

        # Get List of Document Objects from the Output Bucket
        output_blobs = storage_client.list_blobs(output_bucket, prefix=output_prefix)

        # Document AI may output multiple JSON files per source file
        for blob in output_blobs:
            # Document AI should only output JSON files to GCS
            if ".json" not in blob.name:
                print(
                    f"Skipping non-supported file: {blob.name} - Mimetype: {blob.content_type}"
                )
                continue

            # Download JSON File as bytes object and convert to Document Object
            print(f"Fetching {blob.name}")
            document = documentai.Document.from_json(
                blob.download_as_bytes(), ignore_unknown_fields=True
            )

            # For a full list of Document object attributes, please reference this page:
            # https://cloud.google.com/python/docs/reference/documentai/latest/google.cloud.documentai_v1.types.Document

            # Read the text recognition output from the processor
            print("The document contains the following text:")
            print(document.text)


In [21]:
batch_process_documents(keys, gcs_input_uri='localhost://recibos/file.pdf', gcs_output_bucket="localhost://", gcs_output_uri_prefix="home/")

InvalidArgument: 400 Request contains an invalid argument. [field_violations {
  field: "input_documents.gcs_documents.documents[0].gcs_uri"
  description: "Invalid Cloud Storage object URI"
}
field_violations {
  field: "document_output_config.gcs_output_config.gcs_uri"
  description: "Invalid output Cloud Storage URI"
}
]